<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/IMDB-Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [3]:
csv="https://github.com/jgoodall/cinevis/raw/master/data/csvs/moviedata.csv"
csv="https://github.com/shubham1710/Movie-Recommendation-System/raw/master/moviedata.csv"

In [4]:
movie = pd.read_csv(csv)
movie

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

In [5]:
features = ['keywords','cast','genres','director','tagline']
for feature in features:
    movie[feature] = movie[feature].fillna('')
    
def combine_features(row):
    try:
        return row['keywords'] +" "+row['cast']+" "+row['genres']+" "+row['director']+" "+row['tagline']
    except:
        print ("Error:", row)

movie["combined_features"] = movie.apply(combine_features,axis=1)

In [6]:
def title_from_index(index):
    return movie[movie.index == index]["title"].values[0]

def index_from_title(title):
    title_list = movie['title'].tolist()
    common = difflib.get_close_matches(title, title_list, 1)
    titlesim = common[0]
    return movie[movie.title == titlesim]["index"].values[0]

In [7]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(movie["combined_features"])
cosine_sim = cosine_similarity(count_matrix) 

user_movie = input("Enter movie of your choice:\t")
movie_index = index_from_title(user_movie)

similar_movies =  list(enumerate(cosine_sim[movie_index]))
similar_movies_sorted = sorted(similar_movies,key=lambda x:x[1],reverse=True)
i=0
print("\nOther movies you might be interested in:-\n")
for rec_movie in similar_movies_sorted:
        if(i!=0):
            print (i,") ",title_from_index(rec_movie[0]),sep="")
        i=i+1
        if i>50:
            break

Enter movie of your choice:	xmen

Other movies you might be interested in:-

1) Brooklyn
2) Step Up 3D
3) The Longest Ride
4) The Namesake
5) Schindler's List
6) The Lives of Others
7) On The Downlow
8) The White Ribbon
9) World Trade Center
10) The Married Woman
11) The Conspirator
12) Mad City
13) Warcraft
14) Copycat
15) Thor
16) Downfall
17) Mad Max Beyond Thunderdome
18) How to Train Your Dragon
19) Shotgun Stories
20) The Insider
21) Skin Trade
22) Sarah's Key
23) The Girl on the Train
24) The Glimmer Man
25) Adam
26) The Widow of Saint-Pierre
27) Tin Cup
28) Judgment at Nuremberg
29) Jakob the Liar
30) The Devil's Own
31) Weekend
32) Showdown in Little Tokyo
33) The International
34) Sliding Doors
35) You, Me and Dupree
36) Julie & Julia
37) The Work and the Glory
38) The Rock
39) Hart's War
40) Chariots of Fire
41) Undisputed
42) The Wave
43) A Man for All Seasons
44) She's the One
45) Straight Out of Brooklyn
46) Zero Dark Thirty
47) From Paris with Love
48) Hitman
49) One Day

In [29]:
query="jumanji"

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def get_titles_from_imdb(query):
  URL="https://www.imdb.com/find?q="+query+"&ref_=nv_sr_sm"


  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)

  #session.get(url)
  #page = requests.get(URL)
  page = session.get(URL)

  #page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #page = requests.get(URL)
  results = soup.find_all(class_='findSection')
  titles=dict()
  for elem in results:
    h3_elem=elem.find("h3", class_="findSectionHeader")
    section_title=h3_elem.text.strip()
    if(section_title=="Titles"):
      #print(section_title)
      for more_elem in elem.find_all(class_="result_text"):
        # Each job_elem is a new BeautifulSoup object.
        # You can use the same methods on it as you did before.
        #print(more_elem)
        a_elem = more_elem.find('a')
        imdb_id=a_elem["href"].replace("/title/","").replace("/","")
        #print(imdb_id)
        link="https://www.imdb.com"+a_elem["href"]
        full_title=more_elem.text.strip()
        short_title=a_elem.text.strip()
        titles.update({full_title:{"URL":link,"short_title":short_title,"imdb_id":imdb_id}})
  return(titles)

In [51]:
pd.DataFrame(get_titles_from_imdb("john wick:"))

,"John Wick (2014) aka ""John Wick: Sin control""",John Wick: Chapter 3 - Parabellum (2019),John Wick: Chapter 2 (2017)
URL,https://www.imdb.com/title/tt2911666/,https://www.imdb.com/title/tt6146586/,https://www.imdb.com/title/tt4425200/
short_title,John Wick,John Wick: Chapter 3 - Parabellum,John Wick: Chapter 2
imdb_id,tt2911666,tt6146586,tt4425200


In [52]:
titles=get_titles_from_imdb("john wick:")

In [51]:

def cleanText(soup_elem=None):
  if(soup_elem is not None and str(soup_elem) != soup_elem):
    return(soup_elem.text.strip())
  else:
    return(None)

def find_all(soup_elem,tag,decompose_flag=False):
  if(soup_elem is not None):
    if(soup_elem is not None and soup_elem == ""):
      return(soup_elem.find_all(tag))
    else:
      return(None)
  else:
    return(None)
def find(soup_elem,tag=None,decompose_flag=False):
  if(soup_elem is not None):
    if(soup_elem != ""):
      elem=soup_elem.find(tag)
      if(decompose_flag):
        if(elem is not None):
          elem=cleanText(elem)
          soup_elem.decompose()
      return(elem)
    else:
      return(None)
  else:
    return(None)


In [13]:
import pickle
def save_obj(obj, name ):
    with open('/content/drive/Shared drives/2/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('/content/drive/Shared drives/2/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [37]:
import requests
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def get_title_dtl_from_imdb(URL=None,imdb_id=None):
  #URL="https://www.imdb.com/title/tt2911666/"
  BASE_URL="https://www.imdb.com/title/"
  if (URL is None and imdb_id is not None):
    URL=BASE_URL+imdb_id
  if (URL is None):
    return(None)
  if (URL is not None):
    imdb_id=URL.replace(BASE_URL,"").replace("/","")
  



  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)

  #session.get(url)
  #page = requests.get(URL)
  page = session.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #page = requests.get(URL)
  results = soup.find_all(class_='findSection')

  rating_elem = soup.find(class_='ratingValue')
  rating = cleanText(rating_elem)#.text.strip()
  #print(rating)
  reviews=find(rating_elem,"strong")
  if(reviews is not None):
    reviews=reviews["title"]
  #print(reviews)
  title_elem=soup.find(class_='title_wrapper')
  title_year_elem=title_elem.find(id="titleYear")
  year=find(title_year_elem,"a",True)#.text.strip()
  #year=title_year_elem.find("a").text.strip()
  #title_year_elem.decompose()
  title_name=cleanText(title_elem.find("h1"))#.text.strip()
  #print(title_elem)
  more_details_elem=title_elem.find(class_="subtext")
  # get genres
  genres=[]
  for a_elem in more_details_elem.find_all("a"):
    if( "genres" in a_elem["href"]):
      genres.append(cleanText(a_elem))#.text.strip())
      a_elem.decompose()

  #print(genres)

  #movie_time=more_details_elem.find("time").text.strip()
  #more_details_elem.find("time").decompose()

  movie_time=find(more_details_elem,"time")

  #release_info_elem=more_details_elem.find("a")
  #release_info=release_info_elem.text.strip()
  #release_info_elem.decompose()
  
  release_info=cleanText(find(more_details_elem,"a",True))

  #title_elem=title_elem.find()
  #print(more_details_elem.findChildren())
  for span in more_details_elem.find_all("span"):
    span.decompose()
  #maturity_rating=more_details_elem.text.strip().split()[0]
  maturity_rating=cleanText(more_details_elem)
  #print(title_name+":"+maturity_rating)
  if(maturity_rating is not None and maturity_rating !=''):
    maturity_rating=maturity_rating.split()[0]
  #print(more_details)

  title_elem.decompose()

  titles=dict({
        "title":title_name
      , "rating" : rating
      , "reviews":reviews
      , "year":year
      , "genres":genres
      , "movie_time":movie_time
      , "release_info":release_info
      , "maturity_rating":maturity_rating
  })

  credits_elem=soup.find_all(class_="credit_summary_item")

  for creditor in credits_elem:
    creditor_type=creditor.find("h4").text.strip()
    creditor_names=[nm.text.strip() for nm in creditor.find_all("a")]
    titles.update({creditor_type:creditor_names})

  summary_text=cleanText(soup.find(class_="summary_text"))
  titles.update({"summary_text":summary_text})

  title_details_elem=soup.find(id="titleDetails")
  if(title_details_elem is not None):
    for seemore_elems in title_details_elem.find_all(class_="see-more"):
      seemore_elems.decompose()

    #print(credits_elem)
    for title_detail in title_details_elem.find_all(class_="txt-block"):
      title_detail_type=title_detail.find("h4")
      if(title_detail_type is not None):
        title_detail_type1=title_detail_type.text.strip()
        title_detail_type.decompose()
        title_detail_type=title_detail_type1
      #print(title_detail_type)
      title_detail_text1=""
      if (title_detail.find_all("a") is not None):
        title_detail_text1=[nm.text.strip().replace("(estimated)","") for nm in title_detail.find_all("a")]
        [a.decompose() for a in title_detail.find_all("a")]
        [a.decompose() for a in title_detail.find_all(class_="ghost")]
      
      title_detail_text2=title_detail.text.strip()
      #titles.update({creditor_type:creditor_names})
      title_detail_text1.append(title_detail_text2)

      #print({str(title_detail_type).replace(":",""):str(title_detail_text1)})
      titles.update({str(title_detail_type).replace(":",""):str(title_detail_text1)})

  #print(soup.find(id="titleDetails"))

  #print(title_elem)
  return({imdb_id:titles})
  #titles

In [33]:
def searchTitle(title):
  for v in allTitles:
    if(allTitles[v]["title"]==title):
      return(v)
  return(None)

searchTitle("Avatar")

'tt0499549'

In [85]:
titles=get_titles_from_imdb("john wick:")
for title in titles:
  print(titles[title])
  print(get_title_dtl_from_imdb(titles[title]["URL"]))

{'URL': 'https://www.imdb.com/title/tt2911666/', 'short_title': 'John Wick', 'imdb_id': 'tt2911666'}
{'tt2911666': {'title': 'John Wick', 'rating': '7.4/10', 'reviews': '7.4 based on 519,368 user ratings', 'year': '', 'genres': ['Action', 'Crime', 'Thriller'], 'movie_time': <None></None>, 'release_info': None, 'maturity_rating': 'R', 'Directors:': ['Chad Stahelski', 'David Leitch'], 'Writer:': ['Derek Kolstad'], 'Stars:': ['Keanu Reeves', 'Michael Nyqvist', 'Alfie Allen', 'See full cast & crew'], 'summary_text': 'An ex-hit-man comes out of retirement to track down the gangsters that killed his dog and took everything from him.', 'Official Sites': "['Official Facebook', 'Official site', '']", 'Country': "['USA', 'UK', 'China', '']", 'Language': "['English', 'Russian', 'Hungarian', '']", 'Release Date': "['24 October 2014 (USA)']", 'Also Known As': "['John Wick']", 'Filming Locations': "['New York City, New York, USA', '']", 'Budget': "['$20,000,000\\n            (estimated)']", 'Opening

In [86]:
titles

{'John Wick (2014) aka "John Wick: Sin control"': {'URL': 'https://www.imdb.com/title/tt2911666/',
  'imdb_id': 'tt2911666',
  'short_title': 'John Wick'},
 'John Wick: Chapter 2 (2017)': {'URL': 'https://www.imdb.com/title/tt4425200/',
  'imdb_id': 'tt4425200',
  'short_title': 'John Wick: Chapter 2'},
 'John Wick: Chapter 3 - Parabellum (2019)': {'URL': 'https://www.imdb.com/title/tt6146586/',
  'imdb_id': 'tt6146586',
  'short_title': 'John Wick: Chapter 3 - Parabellum'}}

In [52]:
titles=get_titles_from_imdb("Jesus")
for title in titles:
  #print(titles[title])
  print(get_title_dtl_from_imdb(imdb_id=titles[title]["imdb_id"]))
  #print(get_title_dtl_from_imdb())

{'tt0214930': {'title': 'Mary, Mother of Jesus', 'rating': '5.5/10', 'reviews': '5.5 based on 1,169 user ratings', 'year': '1999', 'genres': ['Drama'], 'movie_time': <None></None>, 'release_info': None, 'maturity_rating': '', 'Director:': ['Kevin Connor'], 'Writer:': ['Albert Ross'], 'Stars:': ['Christian Bale', 'Pernilla August', 'Melinda Kinnaman', 'See full cast & crew'], 'summary_text': 'A believable telling of the life of Mary, the chosen by God, mother of Christ. The story follows Mary before conception, at the revealing of the impending birth by the angel Gabrie, and ...\n                    See full summary\xa0»', 'Country': "['USA', '']", 'Language': "['English', '']", 'Release Date': "['14 November 1999 (USA)']", 'Also Known As': "['Mary & Jesus']", 'Filming Locations': "['Budapest, Hungary', '']", 'Production Co': "['HCC Happy Crew Company', 'Hallmark Entertainment', 'The Shriver Family Film Company', ',,']", 'None': "['IMDbPro', 'Show more on\\n  \\xa0»']", 'Runtime': "['60

In [11]:
allTitles=dict()
queriesSent=[]

In [41]:

allTitles=load_obj("allTitles")
queriesSent=load_obj("queriesSent")

In [39]:
#for query in movie.original_title:
#    if(searchTitle(query) is not None):
#      queriesSent.append(query)
#      print(query)
#      continue

In [ ]:
i=1
import random as random
import time

for query in movie.original_title:
  #if(i<20):
    if((i%10)==0):
      save_obj(allTitles,"allTitles")
      save_obj(queriesSent,"queriesSent")
    if(query not in queriesSent):
      #print("Passed: "+query)
      #continue
      print(query)
      titles=get_titles_from_imdb(query)

      for title in titles:
        #print(titles[title])
  
        # Wait for 5 seconds
        time.sleep(random.randint(1, 5))
        if(searchTitle(title) is not None):
          print("Passed title: "+title)
          continue
        if (title not in allTitles):
          allTitles.update(get_title_dtl_from_imdb(imdb_id=titles[title]["imdb_id"]))
        #print(get_title_dtl_from_imdb(imdb_id=titles[title]["imdb_id"]))
        #print(get_title_dtl_from_imdb())
      queriesSent.append(query)
    #i+=1

The Five-Year Engagement
Mr. 3000
The Next Three Days
Ultraviolet
Assault on Precinct 13


In [17]:
print(len(allTitles))
print(len(queriesSent))


8197
1630


In [162]:
save_obj(allTitles,"allTitles")
allTitles2=load_obj("allTitles")
allTitles2
#from IPython.display import FileLinks
#FileLinks('allTitles.pkl')

ValueError: ignored

In [145]:
import json
with open('data.json', 'w') as fp:
    json.dump(str(allTitles), fp)

In [165]:
pd.DataFrame(allTitles).T

,title,rating,reviews,year,genres,movie_time,release_info,maturity_rating,Creators:,Stars:,summary_text,Official Sites,Country,Language,Release Date,Also Known As,Production Co,None,Runtime,Sound Mix,Color,Aspect Ratio,Director:,Writer:,Filming Locations,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Writers:,Star:,Directors:,Creator:
tt0417299,Avatar: The Last Airbender,9.2/10,"9.2 based on 230,166 user ratings",None,"[Animation, Action, Adventure]",[],None,TV-Y7-FV,"[Michael Dante DiMartino, Bryan Konietzko]","[Zach Tyler, Jack De Sena, Dee Bradley Baker, ...","In a war-torn world of elemental magic, a youn...","['Official Facebook', 'Official NL Site', '']","['USA', '']","['English', '']",['21 February 2005 (USA)'],['Nickelodeon Avatar: The Last Airbender'],"['Nickelodeon Animation Studios', '']","['IMDbPro', 'Show more on\n \xa0»']",['23 min'],"['Stereo', '']","['Color', '']",['1.33 : 1'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0499549,Avatar,7.8/10,"7.8 based on 1,100,371 user ratings",2009,"[Action, Adventure, Fantasy]",[],None,PG-13,NaN,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",A paraplegic Marine dispatched to the moon Pan...,"['Official Facebook', 'Official Site', '']","['USA', '']","['English', 'Spanish', '']",['18 December 2009 (USA)'],['Avatar: An IMAX 3D Experience'],"['Twentieth Century Fox', 'Dune Entertainment'...","['IMDbPro', 'Show more on\n \xa0»']",['162 min\n\n171 min\n (special edi...,"['Dolby Digital', 'DTS', 'SDDS', 'Sonics-DDP',...","['Color', '']",['1.78 : 1'],[James Cameron],[James Cameron],"['Chicago Illinois, USA', '']","['$237,000,000\n (estimated)']","['$77,025,481,\n20 December 2009']","['$760,507,625']","['$2,790,439,000']",NaN,NaN,NaN,NaN
tt0449088,Pirates of the Caribbean: At World's End,7.1/10,"7.1 based on 584,777 user ratings",2007,"[Action, Adventure, Fantasy]",[],None,PG-13,NaN,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","Captain Barbossa, Will Turner and Elizabeth Sw...","['Official Facebook', 'Official site', '']","['USA', '']","['English', '']",['25 May 2007 (USA)'],['P.O.T.C. 3'],"['Walt Disney Pictures', 'Jerry Bruckheimer Fi...","['IMDbPro', 'Show more on\n \xa0»']",['169 min\n\n128 min\n (Mainland Ch...,"['SDDS', 'Dolby Digital', 'DTS', 'Cinesound', ...","['Color', '']",['2.39 : 1'],[Gore Verbinski],NaN,"['Nipomo, California, USA', '']","['$300,000,000\n (estimated)']","['$114,732,820,\n27 May 2007']","['$309,420,425']","['$960,996,492']","[Ted Elliott, Terry Rossio, 4 more credits]",NaN,NaN,NaN
tt1043740,Pirates of the Caribbean: At World's End,7.1/10,7.1 based on 959 user ratings,2007,"[Action, Adventure, Crime]",None,None,",",NaN,"[Jack Angel, Lee Arenberg, Julianne Buescher, ...",Hack and slash game following the events of Pi...,NaN,"['USA', '']","['English', 'Spanish', 'Italian', '']",['18 May 2007 (USA)'],"[""Pirates of the Caribbean: At World's End""]","['Walt Disney Productions', '']","['IMDbPro', 'Show more on\n \xa0»']",NaN,NaN,"['Color', '']",NaN,[Nick Bridger],[Michael Humes],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt3327282,Pirates of the Caribbean: At World's End,None,None,None,[Talk-Show],None,None,Episode,NaN,NaN,Add a Plot »,NaN,NaN,NaN,['9 November 2013 (USA)'],NaN,NaN,"['IMDbPro', 'Show more on\n \xa0»']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Raquel Horton],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt11257042,Harry Potter and the Goblet of Fire,None,None,None,[Talk-Show],[],None,2h,NaN,"[Aaron Elliott, Shane Farley, Calvin Grubb, Se...",The crew records a commentary of Harry Potter ...,NaN,"['USA', '']","['English', '']",['16 November 2019 (USA)'],NaN,NaN,"['IMDbPro', 'Show more on\n \xa0»']",['155 min'],NaN,"['Color', '']",['16:9 HD'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt7353728,Harry Potter and the Goblet of Fire/Breakfast ...,None,None,None,[Talk-Show],[],None,TV-PG,NaN,"[Roger Ebert, Richard Roeper]",A